In [2]:
## Graphdb configuration
NEO4J_URI="neo4j+s://ff9960b9.databases.neo4j.io"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="CDmgGKNF41H4y26wukit8TAfc0_Dq2Kc0f1Sq0g9db4"

In [3]:
import os 
os.environ["NEO4J_URI"]=NEO4J_URI
os.environ["NEO4J_USERNAME"]=NEO4J_USERNAME
os.environ["NEO4J_PASSWORD"]=NEO4J_PASSWORD

In [5]:
from langchain_community.graphs import Neo4jGraph
graph=Neo4jGraph(
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
)

In [6]:
graph# connected to graph database

In [7]:
from dotenv import load_dotenv
from langchain_groq import ChatGroq

load_dotenv()
groq_api_key=os.getenv('GROQ_API_KEY')


llm=ChatGroq(groq_api_key=groq_api_key,
             model_name="Gemma2-9b-It")
llm


ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x00000261A09142E0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000261A08E75E0>, model_name='Gemma2-9b-It', groq_api_key=SecretStr('**********'))

In [8]:
from langchain_core.documents import Document
text="""
Ajith Kumar and Thalapathy Vijay are two of the most prominent and celebrated actors in Tamil cinema, often seen as rivals due to their massive fan bases and box office successes. Ajith, fondly known as "Thala," is renowned for his versatility and intense performances, particularly in action-packed roles. With films like "Mankatha," "Vedalam," and "Viswasam," Ajith has established himself as a bankable star, known for his stylish demeanor and strong screen presence. His ability to portray complex characters with ease has earned him critical acclaim and a loyal following.

Thalapathy Vijay, on the other hand, is known for his charismatic screen presence, exceptional dance skills, and mass appeal. With a career spanning over three decades, Vijay has delivered numerous blockbusters, including "Ghilli," "Thuppakki," and "Master." His films often carry a strong social message, resonating with a wide audience. Vijay's performances are marked by his energy and commitment, making him a favorite among fans of all ages.

Both actors have a few films together early in their careers, such as "Rajavin Parvaiyile" and "Rajavin Paarvaiyile," where they shared the screen before they became megastars. Their careers have since evolved independently, creating their own legacies in Tamil cinema.
"""


In [9]:
document=[Document(page_content=text)]

In [10]:
document

[Document(page_content='\nAjith Kumar and Thalapathy Vijay are two of the most prominent and celebrated actors in Tamil cinema, often seen as rivals due to their massive fan bases and box office successes. Ajith, fondly known as "Thala," is renowned for his versatility and intense performances, particularly in action-packed roles. With films like "Mankatha," "Vedalam," and "Viswasam," Ajith has established himself as a bankable star, known for his stylish demeanor and strong screen presence. His ability to portray complex characters with ease has earned him critical acclaim and a loyal following.\n\nThalapathy Vijay, on the other hand, is known for his charismatic screen presence, exceptional dance skills, and mass appeal. With a career spanning over three decades, Vijay has delivered numerous blockbusters, including "Ghilli," "Thuppakki," and "Master." His films often carry a strong social message, resonating with a wide audience. Vijay\'s performances are marked by his energy and com

In [11]:
from langchain_experimental.graph_transformers import LLMGraphTransformer
llm_transformer=LLMGraphTransformer(llm=llm)

In [12]:
graph_documents=llm_transformer.convert_to_graph_documents(document)

In [13]:
graph_documents

[GraphDocument(nodes=[Node(id='Ajith Kumar', type='Person'), Node(id='Thalapathy Vijay', type='Person')], relationships=[Relationship(source=Node(id='Ajith Kumar', type='Person'), target=Node(id='Thalapathy Vijay', type='Person'), type='COSTAR'), Relationship(source=Node(id='Ajith Kumar', type='Person'), target=Node(id='Person', type='Person'), type='RIVAL'), Relationship(source=Node(id='Thalapathy Vijay', type='Person'), target=Node(id='Person', type='Person'), type='RIVAL')], source=Document(page_content='\nAjith Kumar and Thalapathy Vijay are two of the most prominent and celebrated actors in Tamil cinema, often seen as rivals due to their massive fan bases and box office successes. Ajith, fondly known as "Thala," is renowned for his versatility and intense performances, particularly in action-packed roles. With films like "Mankatha," "Vedalam," and "Viswasam," Ajith has established himself as a bankable star, known for his stylish demeanor and strong screen presence. His ability to

In [14]:
graph_documents[0].nodes

[Node(id='Ajith Kumar', type='Person'),
 Node(id='Thalapathy Vijay', type='Person')]

In [15]:
graph_documents[0].relationships

[Relationship(source=Node(id='Ajith Kumar', type='Person'), target=Node(id='Thalapathy Vijay', type='Person'), type='COSTAR'),
 Relationship(source=Node(id='Ajith Kumar', type='Person'), target=Node(id='Person', type='Person'), type='RIVAL'),
 Relationship(source=Node(id='Thalapathy Vijay', type='Person'), target=Node(id='Person', type='Person'), type='RIVAL')]

In [16]:

### Load the dataset of movie

movie_query="""
LOAD CSV WITH HEADERS FROM
'https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/movies/movies_small.csv' as row

MERGE(m:Movie{id:row.movieId})
SET m.released = date(row.released),
    m.title = row.title,
    m.imdbRating = toFloat(row.imdbRating)
FOREACH (director in split(row.director, '|') |
    MERGE (p:Person {name:trim(director)})
    MERGE (p)-[:DIRECTED]->(m))
FOREACH (actor in split(row.actors, '|') |
    MERGE (p:Person {name:trim(actor)})
    MERGE (p)-[:ACTED_IN]->(m))
FOREACH (genre in split(row.genres, '|') |
    MERGE (g:Genre {name:trim(genre)})
    MERGE (m)-[:IN_GENRE]->(g))
"""

In [17]:
graph 

In [18]:
graph.query(movie_query)

[]

In [19]:
graph.refresh_schema()
print(graph.schema)

Node properties:
Person {born: INTEGER, name: STRING, profession: STRING}
Movie {title: STRING, released: INTEGER, id: STRING, imdbRating: FLOAT, actors: LIST, director: LIST, genres: LIST}
Genre {name: STRING}
Director {name: STRING}
Actor {name: STRING}
Relationship properties:

The relationships:
(:Person)-[:ACTED_IN]->(:Movie)
(:Person)-[:DIRECTED]->(:Movie)
(:Movie)-[:IN_GENRE]->(:Genre)


In [20]:
from langchain.chains import GraphCypherQAChain
chain=GraphCypherQAChain.from_llm(llm=llm,graph=graph,verbose=True)
chain

GraphCypherQAChain(verbose=True, graph=<langchain_community.graphs.neo4j_graph.Neo4jGraph object at 0x000002619FCE95A0>, cypher_generation_chain=LLMChain(prompt=PromptTemplate(input_variables=['question', 'schema'], template='Task:Generate Cypher statement to query a graph database.\nInstructions:\nUse only the provided relationship types and properties in the schema.\nDo not use any other relationship types or properties that are not provided.\nSchema:\n{schema}\nNote: Do not include any explanations or apologies in your responses.\nDo not respond to any questions that might ask anything else than for you to construct a Cypher statement.\nDo not include any text except the generated Cypher statement.\n\nThe question is:\n{question}'), llm=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x00000261A09142E0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000261A08E75E0>, model_name='Gemma2-9b-It', groq_api_key=SecretStr('**********'))

In [22]:
response=chain.invoke({'query':'Who was actors of movie GoldenEye'})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (m:Movie {title: "GoldenEye"})<-[:ACTED_IN]-(a:Person)
RETURN a.name

Full Context:
[{'a.name': 'Pierce Brosnan'}, {'a.name': 'Famke Janssen'}, {'a.name': 'Sean Bean'}, {'a.name': 'Izabella Scorupco'}]

> Finished chain.


{'query': 'Who was actors of movie GoldenEye',
 'result': 'Pierce Brosnan, Famke Janssen, Sean Bean, Izabella Scorupco  \n'}

In [23]:
response=chain.invoke({'query':'Top 5 imdb rating movies'})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie)
RETURN m.title, m.imdbRating
ORDER BY m.imdbRating DESC
LIMIT 5 

Full Context:
[{'m.title': 'Billa', 'm.imdbRating': None}, {'m.title': 'Loe', 'm.imdbRating': None}, {'m.title': 'Shawshank Redemption, The', 'm.imdbRating': 9.3}, {'m.title': 'Shawshank Redemption, The', 'm.imdbRating': 9.3}, {'m.title': 'Pulp Fiction', 'm.imdbRating': 8.9}]

> Finished chain.


{'query': 'Top 5 imdb rating movies',
 'result': 'Shawshank Redemption, The, Pulp Fiction \n'}